## DHK Airdrop Script

In [9]:
# These are parameters used in the program below
from datetime import datetime, timezone
from dotenv import load_dotenv
from os import environ

load_dotenv()

DHK_DISTRIBUTION = 100000
# Date for the price reference
PRICE_REF_DATE = datetime(2024, 7, 29, 12, tzinfo=timezone.utc)

TOKENS = [
    { "token": "AKT", "network": "akash", "qty": 189628 },
    { "token": "ATOM", "network": "cosmos", "qty": 592191 },

    # manual price and staking-apr
    { "token": "DSM", "price": 0.002867, "staking-apr": 0, "qty": 3967661 },
    { "token": "EVMOS", "network": "evmos", "qty": 248824 },
    { "token": "JUNO", "network": "juno", "qty": 87290 },
    { "token": "OSMO", "network": "osmosis", "qty": 984661 },

    # manual price and staking-apr
    { "token": "STARS", "price": 0.00717, "staking-apr": 0.1355, "qty": 1908776 },
]

APIS = {
    "cryptocompare": {
        "endpoint": "https://min-api.cryptocompare.com/data/pricehistorical",
        "apikey": environ.get('CRYPTOCOMPARE_APIKEY'),
    },
    "mintscan": {
        "endpoint": "https://apis.mintscan.io/v1/:network/apr",
        "apikey": environ.get('MINTSCAN_APIKEY'),
    },
}

In [10]:
# Program to retrieve necessary token information for export
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import math
import random
import pandas as pd

# API doc: https://min-api.cryptocompare.com/documentation?key=Historical&cat=dataPriceHistorical
def fetch_price(from_symbol, to_symbol='USD', date=PRICE_REF_DATE):
    endpoint, apikey = APIS["cryptocompare"]["endpoint"], APIS["cryptocompare"]["apikey"]
    parameters = {
        'fsym':from_symbol,
        'tsyms': to_symbol,
        'calculationType':'MidHighLow',
        'ts': date.timestamp()
    }
    headers = {
        'Accepts': 'application/json',
        'authorization': f"Apikey {apikey}",
    }

    session = Session()
    session.headers.update(headers)
    try:
        response = session.get(endpoint, params=parameters)
        data = json.loads(response.text)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        raise Exception(f"fetch_price connection error: {e}")

    if(from_symbol not in data):
        raise Exception(f"unable to fetch price, returning: {data}")
    
    return data[from_symbol][to_symbol]    


# API doc: https://docs.cosmostation.io/apis/reference/utilities/staking-apr
def fetch_staking_apr(network):
    endpoint, apikey = APIS["mintscan"]["endpoint"], APIS["mintscan"]["apikey"]

    endpoint = endpoint.replace(":network", network)
    headers = {
        'Accepts': 'application/json',
        'authorization': f"Bearer {apikey}",
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(endpoint)
        data = json.loads(response.text)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        raise Exception(f"fetch_staking_apr connection error: {e}")

    if("apr" not in data):
        raise Exception(f"unable to fetch staking_apr, returning: {data}")
    
    return float(data["apr"])
    

def get_main_table():
    columns = [
        "token", "price", "staking-amt", "staking-val", "staking-apr", 
        "reward", "dhk-distribution-pc", "dhk-distribution"
    ]
    lst = []
    
    for t in TOKENS:
        token, staking_amt = t["token"], t["qty"]
        token_price = t["price"] if "price" in t else round(fetch_price(token), 5)
        
        staking_val = round(token_price * staking_amt, 2)
        staking_apr = 0
        if "staking-apr" in t:
            staking_apr = t["staking-apr"]
        elif "network" in t:
            staking_apr = round(fetch_staking_apr(t["network"]), 4)
        
        reward = round(staking_apr * staking_val, 2)

        lst.append([token, token_price, staking_amt, staking_val, staking_apr, reward, 0, 0])

    mt = pd.DataFrame(lst, columns=columns)
    
    # Append a total row at the end of the table
    ttl_staking_val = mt["staking-val"].sum()
    ttl_reward = mt["reward"].sum()
    ttl = pd.Series({"token": "TOTAL", "staking-val": ttl_staking_val, "reward": ttl_reward})
    mt = pd.concat([mt, ttl.to_frame().T], ignore_index=True)

    # Calculate DHK distribution and distribution percent
    for idx, row in mt.iterrows():
        mt.at[idx, "dhk-distribution-pc"] = round((row["reward"] / ttl_reward) * 100, 3)
        mt.at[idx, "dhk-distribution"] = math.floor(row["reward"] / ttl_reward * DHK_DISTRIBUTION)
    
    return mt


get_main_table()

,token,price,staking-amt,staking-val,staking-apr,reward,dhk-distribution-pc,dhk-distribution
0,AKT,3.225500,189628.0,611645.11,0.1434,87709.91,12.721,12721.0
1,ATOM,6.238000,592191.0,3694087.46,0.1507,556698.98,80.741,80740.0
2,DSM,0.002867,3967661.0,11375.28,0.0000,0.0,0.000,0.0
3,EVMOS,0.029390,248824.0,7312.94,0.0219,160.15,0.023,23.0
4,JUNO,0.108500,87290.0,9470.97,0.1782,1687.73,0.245,244.0
5,OSMO,0.485800,984661.0,478348.31,0.0865,41377.13,6.001,6001.0
6,STARS,0.007170,1908776.0,13685.92,0.1355,1854.44,0.269,268.0
7,TOTAL,NaN,NaN,4825925.99,NaN,689488.34,100.000,100000.0
